In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import zarr
import holoviews as hv
import matplotlib.pyplot as plt
from functools import partial
from cytoolz import *
import hvplot.pandas
import scipy

IDX = pd.IndexSlice

# Traces

In [ ]:
nd2_filename = (
    "/home/jqs1/scratch/190504/basilisk/190504_YFP_mScarlet_repressilators002.nd2"
)

In [ ]:
dat = pq.read_pandas(
    nd2_filename + ".measurements/pos177.labelwise.parquet"
).to_pandas()

In [ ]:
dat.head()

In [ ]:
dat.loc[IDX[1, 62, :, 1], ("YFP", "noerode", "mean")].rename("h").reset_index().hvplot(
    "t", "h"
) * dat.loc[IDX[1, 62, :, 1], ("mCherry", "noerode", "mean")].rename(
    "h"
).reset_index().hvplot(
    "t", "h"
)

## Segmentation

In [ ]:
root = zarr.open_group(
    store=zarr.LMDBStore(nd2_filename + ".images/pos176.zarr", readonly=True)
)

In [ ]:
trench_set = 1
trench_idx = 62
raw_stack = np.asarray(root["raw"][trench_set][trench_idx]["CFP"])
label_stack = np.asarray(root["segmentation"][trench_set][trench_idx]["CFP"])

In [ ]:
raw_stack.shape

In [ ]:
hv.Image(raw_stack[:, :, 14])

In [ ]:
hv.HoloMap({t: hv.Image(np.random.random((10, 10))) for t in range(2)})

In [ ]:
%%output size=200
hv.HoloMap({t: hv.Image(raw_stack[:, :, t]) for t in range(raw_stack.shape[-1] // 10)})

In [ ]:
%%opts Image {+framewise}
hv.HoloMap(
    {t: hv.Image(raw_stack[:, :, t]) for t in range(raw_stack.shape[-1])}
) + hv.HoloMap({t: hv.Image(label_stack[:, :, t]) for t in range(raw_stack.shape[-1])})